In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from scipy.integrate import odeint
import random

In [2]:
def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x*(rho - z) - y
    dzdt = x*y - beta*z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(initial_state=[1.0, 1.0, 1.0], tmax=250, dt=0.02,
                         sigma=10.0, rho=28.0, beta=8.0/3.0):
    num_steps = int(tmax / dt) + 1
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

In [3]:
class SequenceDataset(Dataset):
    def __init__(self, data, input_len=50, pred_len=1000):
        self.input_len = input_len
        self.pred_len = pred_len
        self.data = data
        self.X, self.Y = self.create_sequences()

    def create_sequences(self):
        X, Y = [], []
        total_len = self.input_len + self.pred_len
        for i in range(len(self.data) - total_len):
            X.append(self.data[i:i+self.input_len])
            Y.append(self.data[i+self.input_len:i+self.input_len+self.pred_len])
        return torch.tensor(np.array(X), dtype=torch.float32), torch.tensor(np.array(Y), dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [4]:
class LSTMForecaster(nn.Module):
    def __init__(self, input_size=3, hidden_size=300, num_layers=1, dropout=0.0):
        super(LSTMForecaster, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x, steps):
        h, _ = self.lstm(x)
        h_last = h[:, -1:]
        preds = []
        out = self.linear(h_last)
        preds.append(out)

        for _ in range(steps - 1):
            h_next, _ = self.lstm(out)
            out = self.linear(h_next)
            preds.append(out)

        return torch.cat(preds, dim=1)


In [5]:
def nrmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2)) / (np.max(y_true) - np.min(y_true))

# Train function
def train_model(model, train_loader, epochs=10, lr=1e-3):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    for epoch in range(epochs):
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            preds = model(x_batch, y_batch.shape[1])
            loss = loss_fn(preds, y_batch)
            loss.backward()
            optimizer.step()

In [6]:
# Evaluation at different horizons
def evaluate_model(model, test_loader, horizons):
    model.eval()
    errors = {h: [] for h in horizons}
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            pred = model(x_batch, max(horizons))
            for h in horizons:
                true = y_batch[:, :h].numpy()
                est = pred[:, :h].numpy()
                for t, p in zip(true, est):
                    errors[h].append(nrmse(t, p))
    return {h: (np.mean(errors[h]), np.std(errors[h])) for h in horizons}

In [7]:
from itertools import product

def grid_search_configs():
    hidden_sizes = [300]
    lrs = [1e-2, 1e-3]
    batch_sizes = [32, 64]
    num_layers_list = [1, 2]
    dropouts = [0.0, 0.2]
    return list(product(hidden_sizes, lrs, batch_sizes, num_layers_list, dropouts))

def run_grid_search():
    results = {}
    print_horizons = list(range(200, 1001, 200))
    all_horizons = list(range(10, 1001, 10))

    for split_ratio in [0.75, 0.8]:
        for (hidden_size, lr, batch_size, num_layers, dropout) in grid_search_configs():
            config_key = f"Split_{split_ratio}_H{hidden_size}_LR{lr}_B{batch_size}_L{num_layers}_D{dropout}"
            results[config_key] = {h: [] for h in print_horizons}

            for seed in range(10):
                np.random.seed(seed)
                torch.manual_seed(seed)
                random.seed(seed)

                for initial_state in [[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [2.0, 1.5, 4.0]]:
                    data_raw = generate_lorenz_data(initial_state=initial_state)[1]
                    scaler = MinMaxScaler()
                    data_scaled = scaler.fit_transform(data_raw)

                    split_idx = int(len(data_scaled) * split_ratio)
                    train_data = data_scaled[:split_idx]
                    test_data = data_scaled[split_idx - 1050:]  # to ensure long enough sequence

                    train_dataset = SequenceDataset(train_data)
                    test_dataset = SequenceDataset(test_data)

                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    test_loader = DataLoader(test_dataset, batch_size=batch_size)

                    model = LSTMForecaster(hidden_size=hidden_size, num_layers=num_layers, dropout=dropout)
                    train_model(model, train_loader, epochs=10, lr=lr)

                    metrics = evaluate_model(model, test_loader, all_horizons)

                    for h in print_horizons:
                        results[config_key][h].append(metrics[h][0])  # mean NRMSE at horizon h

            # Mean ± Std Dev across seeds and initial states
            for h in print_horizons:
                vals = results[config_key][h]
                mean = np.mean(vals)
                std = np.std(vals)
                results[config_key][h] = f"{mean:.4f} ± {std:.4f}"

            print(f"✅ Done: {config_key}")

    return results


In [8]:
import pandas as pd

def save_results_to_csv(results_dict, filename="grid_results.csv"):
    rows = []
    for config, horizon_dict in results_dict.items():
        for h, val in horizon_dict.items():
            mean_str, std_str = val.split(" ± ")
            rows.append({
                "Config": config,
                "Horizon": h,
                "NRMSE_Mean": float(mean_str),
                "NRMSE_Std": float(std_str)
            })
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False)
    print(f"Saved results to {filename}")
    return df


def get_best_config(results_df, horizon=1000):
    df_h = results_df[results_df['Horizon'] == horizon]
    best_row = df_h.loc[df_h['NRMSE_Mean'].idxmin()]
    print("\n✅ Best Config Based on NRMSE @ Horizon", horizon)
    print(best_row)
    return best_row


In [ ]:
results = run_grid_search()
df = save_results_to_csv(results, "lstm_lorenz_results.csv")
best = get_best_config(df)